In [ ]:
#!pip install Mediapipe
#!pip install protobuf==3.20.*
#!pip install opencv-python


In [ ]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings

from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from skimage import io

warnings.filterwarnings('ignore')

In [ ]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy


    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [ ]:
mylist = [Path(f).stem for f in glob.glob(r"C:\Users\user\Downloads\AFLW2000/*.mat")]

In [ ]:
file_path = r"C:\Users\user\Downloads\AFLW2000"
paths = []
for i in range(len(mylist)):
    mat_path = os.path.join(file_path,mylist[i] + ".mat")
    paths.append(mat_path)

paths


In [ ]:
def get_ypr_from_mat(mat_path):
    mat = sio.loadmat(mat_path)
    pre_pose_params = mat['Pose_Para'][0]
    pose_params = pre_pose_params[:3]
    return pose_params

ypr_list = []
for index in range(len(paths)):
    ypr_list.append(get_ypr_from_mat(paths[index]))

ypr_df = pd.DataFrame(ypr_list,columns=["yaw","pitch","roll"])
ypr_df['img'] = mylist
ypr_df

In [ ]:
x_relative = []
y_relative = []
img = []
for iter in mylist:
    faceModule = mediapipe.solutions.face_mesh
    with faceModule.FaceMesh(static_image_mode=True) as faces:
        image = io.imread(r'C:\Users\user\Downloads\AFLW2000/'+iter+'.jpg')
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None: 
            for face in results.multi_face_landmarks:
                lst1 = []
                lst2 = []
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y
                    shape = image.shape 
                    relative_x = int(x * shape[1])
                    relative_y = int(y * shape[0])
                    cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
                    lst1.append(relative_x)
                    lst2.append(relative_y)

                x_relative.append(lst1)
                y_relative.append(lst2)
                img.append(iter)

In [ ]:
x_df = pd.DataFrame(x_relative)
y_df = pd.DataFrame(y_relative)
img_df = pd.DataFrame(img , columns=['img'])

In [ ]:
concatenated_dfs = pd.concat([x_df, y_df,img_df], axis="columns")

In [ ]:
concatenated_dfs

In [ ]:
dfff=concatenated_dfs.join(ypr_df.set_index('img'), on='img')

In [ ]:
dfff.to_csv("my_data",index=False)

In [ ]:
my_data = pd.read_csv(r"C:\Users\user\my_data")
my_data

In [ ]:
x= my_data.drop(['yaw'	,'pitch'	,'roll','img'],axis=1)
y= my_data[['yaw'	,'pitch'	,'roll']]

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
regr = MultiOutputRegressor(SVR(C=3,epsilon=0.02,tol=0.00001))

pipe = Pipeline([('scaler', StandardScaler()),('pca',PCA(n_components=500)), ('svr', regr)])

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
pipe.score(X_test, y_test)

In [ ]:
y_perd=pipe.predict(X_test)

In [ ]:
print("R2 score: ", r2_score(y_test, y_perd))
print("MSE: ", mean_squared_error(y_test, y_perd))

In [ ]:
import pickle

filename = 'finalized_model.sav'
pickle.dump(pipe, open(filename, 'wb'))


In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))


In [ ]:
image = io.imread(r'C:\Users\user\Downloads\AFLW2000/'+'image00079'+'.jpg')
mat_file = sio.loadmat(r'C:\Users\user\Downloads\AFLW2000/'+'image00079'+'.mat')
pose_para = mat_file["Pose_Para"][0][:3]
pitch = pose_para[0]
yaw = pose_para[1]
roll = pose_para[2]
plt.imshow(draw_axis(image,pitch,yaw,roll))

In [ ]:
def getData(image,draw_mesh=False):
    faceModule = mediapipe.solutions.face_mesh
    x_relative = []
    y_relative = []
    images_data = []
    mesh_image = image.copy()

    with faceModule.FaceMesh(static_image_mode=True) as faces:
        image_data = []
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        if results.multi_face_landmarks != None:
            for face in results.multi_face_landmarks:
                lst1=[]
                lst2=[]
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y
                    shape = image.shape 
                    relative_x = int(x * shape[1])
                    relative_y = int(y * shape[0])
                    lst1.append(relative_x)
                    lst2.append(relative_y)
                    if draw_mesh:
                        cv2.circle(mesh_image, (relative_x, relative_y), radius=1, color=(0, 200, 0), thickness=2)
                    
                x_relative.append(lst1)
                y_relative.append(lst2)
     
    df1 = pd.DataFrame(x_relative)
    df2 = pd.DataFrame(y_relative)
    df =  pd.concat([df1,df2],axis="columns")
    cols = []
    cols += [x+str(i+1) for i in range(468) for x in ['X']]
    cols += [x+str(i+1) for i in range(468) for x in ['Y']]
    df.columns = cols
    return df, image, mesh_image

In [ ]:
cols = []
cols += [x+str(i+1) for i in range(468) for x in ['X']]
cols += [x+str(i+1) for i in range(468) for x in ['Y']]

cols

In [ ]:
def functionality(frame):
    df_frame, frame,frame_mesh = getData(frame, draw_mesh=False)
    if df_frame.size > 0:
        df_frame, image, mesh = getData(frame)
        y_pred = pipe.predict(df_frame)[0]
        pitch = y_pred[0]
        yaw = y_pred[1]
        roll = y_pred[2]
        tdx = int(df_frame['X3'].values)
        tdy = int(df_frame['Y3'].values)
        print(y_pred)
        return draw_axis(frame_mesh,pitch,yaw,roll, tdx=tdx, tdy=tdy, size=50)
    else:
        return frame
    


In [ ]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    c = cv2.waitKey(1) 
    ret, frame = cap.read()
    
    if frame is not None and ret:
            frame = functionality(frame)

    
    frame = cv2.resize(frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)

    cv2.imshow('Pose Detection Application', frame)
    
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()